In [1]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

In [2]:
model = AutoModelForQuestionAnswering.from_pretrained('./models/test_all_model')
tokenizer = AutoTokenizer.from_pretrained('./models/test_all_model')


In [3]:
def answer_question_v2(question, context, tokenizer):
    inputs = tokenizer(
        question, context,
        return_tensors="pt", truncation=True, max_length=512
    )
    predicted_answer = ""
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        start_index = torch.argmax(start_logits)
        end_index = torch.argmax(end_logits) + 1

        predicted_answer = tokenizer.decode(inputs["input_ids"][0][start_index:end_index]).lower()
    return predicted_answer 


In [4]:
from datasets import load_dataset

squad_dataset = load_dataset('squad')

In [5]:
# Select a sample from the validation set
sample = squad_dataset['validation'][2]
question = sample['question']
context = sample['context']

print(f"Given context: {context}")
print(f"Question: {question}")
print(f"Answer: {answer_question_v2(question, context, tokenizer)}")
print(f"True Answer: {sample['answers']}")

Given context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Question: Where did Super Bowl 50 take place?
Answer: levi ' s stadium
True Answer: {'text': ['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, Californ